เราจะวัดค่าความถูกต้องของโมเดลโดบการให้ LLM judge คำตอบที่เราได้จาก RAG เเละ LLM อีกตัว

เชื่อม google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


โหลดสิ่งที่จำเป็น

In [ ]:
!pip install -U langchain-community
# ติดตั้ง PyTorch เวอร์ชันล่าสุด
!pip install torch==2.0.1
# 🚀 ลบ PyTorch และ Transformers เก่าทั้งหมด
!pip uninstall -y torch torchvision torchaudio transformers

# 🚀 ติดตั้ง PyTorch ที่รองรับ CUDA ของ Colab (CUDA 11.8)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 🚀 ติดตั้ง Transformers เวอร์ชันที่รองรับ
!pip install transformers --upgrade


สร้าง Evaluation dataset ภาษาอังกฤษ

In [ ]:
import json
import time
import requests

# 🔑 API Key สำหรับ Together
api_key = "xxxx"  # 👈 ใส่ Together API Key ของคุณตรงนี้

# 🚀 โหลดไฟล์คำแนะนำพร้อมหมวดหมู่
embedding_file = '/content/drive/MyDrive/LockLearn/recommendations_with_categories.json'
output_file = '/content/drive/MyDrive/LockLearn/evaluation_datasetv2.json'

with open(embedding_file, 'r', encoding='utf-8') as f:
    recommendations_with_categories = json.load(f)

# 🚀 ฟังก์ชันเรียก API เพื่อสร้างคำถาม
def request_with_retry(prompt, retries=3, delay=10):  # เรียกทีละคำขอ
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    json_data = {
        "model": "meta-llama/llama-4-scout-17b-16e-instruct",
        "messages": [{"role": "user", "content": prompt}]
    }

    for attempt in range(retries):
        try:
            response = requests.post("https://api.together.xyz/v1/chat/completions", headers=headers, json=json_data, timeout=30)
            if response.status_code == 200:
                return response.json().get("choices", [])[0]["message"]["content"]
            elif response.status_code == 429:  # ถ้าถูก rate-limited
                print("❌ Rate limit reached. Sleeping for 60 seconds...")
                time.sleep(60)  # หน่วง 60 วินาทีถ้าโดน rate limit
            else:
                print(f"❌ Error (Status {response.status_code}): {response.text}")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"❌ Connection Error: {e}")
            time.sleep(delay)
    return None

# 🚀 กรองคำแนะนำตามหมวดหมู่
category_dict = {}
for rec in recommendations_with_categories:
    category = rec.get('category', 'Unknown')
    category_dict.setdefault(category, []).append(rec)

# 🚀 เลือกคำแนะนำจากแต่ละหมวดหมู่ (สูงสุด 100)
filtered_recommendations = [rec for recs in category_dict.values() for rec in (recs if len(recs) <= 100 else random.sample(recs, 100))]

# 🚀 สร้างคำถามจากคำแนะนำที่เลือกมา
evaluation_data = []
print(f"✅ รวมคำแนะนำที่เลือกทั้งหมด: {len(filtered_recommendations)}")

for i, rec in enumerate(filtered_recommendations):
    # ปรับ prompt ให้ LLM สร้างคำถามแบบกระชับเพียงคำถามเดียวต่อคำแนะนำ
    prompt = f"Generate a short, human-like question based on the following recommendation: {rec['text']}"

    print(f"🚀 กำลังสร้างคำถามที่ {i + 1}")
    question = request_with_retry(prompt, delay=10)  # เรียกทีละคำขอ

    if question:
        entry = {"question": question.strip(), "answer": rec['text'], "category": rec.get('category', 'Unknown')}
        evaluation_data.append(entry)

    # หน่วง 1 วินาทีหลังจากแต่ละคำขอ
    time.sleep(1)

# 🚀 บันทึกผลลัพธ์แบบ JSON Pretty
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(evaluation_data, f, ensure_ascii=False, indent=4)

print(f"✅ เสร็จสิ้นการสร้างคำถามทั้งหมด: {len(evaluation_data)}")

# ✅ ตรวจสอบว่าจำนวนคำถามครบ 1,370 คำถามหรือไม่
if len(evaluation_data) == 1370:
    print("✅ คำถามครบ 1,370 คำถามแล้ว!")
else:
    print(f"❌ พบปัญหา: จำนวนคำถามที่ได้มี {len(evaluation_data)} คำถามจาก 1,370 คำถามที่คาดไว้")
